# 转换 API 服务到 MCP 服务

AgentKit 网关支持将现有 API 服务一键转换为 MCP 服务。通过上传 Swagger 或 OpenAPI 3 格式的 API 文档，AgentKit 网关支持批量将存量系统的 API 服务转换为 MCP 服务。在这一过程中，你无需改动任何代码，AgentKit 可以帮助你快速提升大模型与存量系统的交互能力。部署 MCP 服务时，AgentKit 网关可以访问你的函数服务中的 API 服务，也可以通过固定域名的形式访问外部的 API 服务。AgentKit 网关支持入站认证和出站认证，在认证入站请求的同时，为出站请求添加身份凭据，兼容多种系统的认证策略。

在本教程中，我们将演示如何将 API 服务转换为 MCP 服务，你将学会

1. 如何在 AgentKit 网关中将 API 服务转换为 MCP 服务。

在开始本教程之前，请确保你已经**正确配置了凭证**。

> **注意**：如果你还没有配置凭证，请先学习 [`01-tutorials/01-runtime/00_configuration_and_credentials.ipynb`](../01-runtime/00_configuration_and_credentials.ipynb) 教程完成配置。

## 1. 转换 API 服务到 MCP 服务

AgentKit 网关支持通过 Swagger 或 OpenAPI 3 格式的 API 文档，将存量系统的 API 服务转换为 MCP服务。你可以选择函数服务或者固定域名类型的 API 服务，通过 AgentKit 网关转换为 MCP 服务。当 MCP 请求抵达 AgentKit 网关时，AgentKit 网关将判断 MCP 请求的类型主动回复响应或是转换请求交由后端处理。对于 MCP `tools/list` 请求，AgentKit 网关将直接返回 API 定义中的各种操作。对于 MCP `tools/call` 请求，AgentKit 网关将 MCP 请求中的报文转换为 API 请求并发送到后端服务器中。

AgentKit 网关支持从 MCP 请求中将请求内容转换为 API 请求的 URL 参数（Query）、请求头、Cookie，以及表单（Form）或 JSON 类型的请求体。AgentKit 网关支持从 JSON 类型的 API 响应中响应体转换为 MCP 响应的响应内容。你可以阅读[如何上传正确的 MCP 配置文件](https://www.volcengine.com/docs/6569/1816086#OA4azxgS)获得更多关于服务转换的条件和细节。

[Postman Echo](https://postman-echo.com/) 是 Postman 提供的样例应用，它提供了一组用于测试 HTTP 客户端的调试 API。例如，当你通过 `curl` 调用 GET Request 接口时，你将会收到对应的返回。

In [ ]:
!curl "https://postman-echo.com/get"

我们还需要为 Postman Echo 服务准备 Swagger 或 OpenAPI 3 格式的 API 文档。

如果你有 Postman Collection，通过 [Postman to OpenAPI](https://www.postman.com/postman/postman-public-workspace/collection/1blkypo/postman-to-openapi) 可以方便的从 Postman Collection 转换为 OpenAPI 3 文档。如果你的应用通过 `Spring Boot`、`NestJS` 和 `FastAPI` 等框架构建，你也可以通过相关工具快速生成 Swagger 或 OpenAPI 3 文档。

最终，我们得到了 Postman Echo 服务的 OpenAPI 3 文档（为了方便阅读本教程，我们对一部分内容进行了裁剪）。AgentKit 网关支持 JSON 和 YAML 格式的 API 文档。

In [ ]:
api_config = """
openapi: 3.0.3
info:
  title: Postman Echo
  description: >-
    Postman Echo is a service you can use to test your REST clients and make
    sample API calls. It provides endpoints for `GET`, `POST`, `PUT`, various
    auth mechanisms and other utility endpoints.


    The documentation for the endpoints as well as example responses can be
    found at
    [https://postman-echo.com](https://postman-echo.com/?source=echo-collection-app-onboarding)
  version: 1.0.0
servers:
  - url: https://postman-echo.com
  - url: postman-echo.com
paths:
  /get:
    get:
      summary: GET Request
      description: >-
        The HTTP `GET` request method is meant to retrieve data from a server.
        The data

        is identified by a unique URI (Uniform Resource Identifier).


        A `GET` request can pass parameters to the server using "Query String

        Parameters". For example, in the following request,


        >
        [http://example.com/hi/there?hand=wave](http://example.com/hi/there?hand=wave)


        The parameter "hand" has the value "wave".


        This endpoint echoes the HTTP headers, request parameters and the
        complete

        URI requested.
      operationId: getRequest
      parameters:
        - name: foo1
          in: query
          schema:
            type: string
            example: bar1
        - name: foo2
          in: query
          schema:
            type: string
            example: bar2
      responses:
        '200':
          description: GET Request Woops
          content:
            application/json:
              schema:
                type: object
                properties:
                  args:
                    type: object
                    properties:
                      foo1:
                        type: string
                        example: bar1
                      foo2:
                        type: string
                        example: bar2
                  headers:
                    type: object
                  url:
                    type: string
"""

接下来，我们将为 Swagger Petstore 服务创建 MCP 服务。

In [ ]:
from agentkit.sdk.mcp.client import AgentkitMCPClient
from agentkit.sdk.mcp.types import (
    CreateMCPServiceRequest,
    ProtocolForCreateMCPService,
    ProtocolHttpApiForCreateMCPService,
    NetworkForCreateMCPService,
    InboundAuthorizerForCreateMCPService,
    InboundAuthorizerAuthorizerForCreateMCPService,
    InboundAuthorizerAuthorizerKeyAuthForCreateMCPService,
    InboundAuthorizerAuthorizerKeyAuthApiKeysItemForCreateMCPService,
    BackendForCreateMCPService,
    BackendCustomForCreateMCPService,
)
import base64
import uuid

id = str(uuid.uuid4())[-8:]

client = AgentkitMCPClient()
request = CreateMCPServiceRequest(
    Name=f"mcp-service-{id}",
    # 转换 API 服务到 MCP 服务时，`ProtocolType` 为 `HTTP`。
    ProtocolType="HTTP",
    ProtocolConfiguration=ProtocolForCreateMCPService(
        HttpApiConfiguration=ProtocolHttpApiForCreateMCPService(
            Configuration=f"{base64.b64encode(api_config.encode()).decode()}",
        )
    ),
    Path="/mcp",
    NetworkConfiguration=NetworkForCreateMCPService(
        EnablePublicNetwork=True,
        EnablePrivateNetwork=False,
    ),
    InboundAuthorizerConfiguration=InboundAuthorizerForCreateMCPService(
        AuthorizerType="ApiKey",
        Authorizer=InboundAuthorizerAuthorizerForCreateMCPService(
            KeyAuth=InboundAuthorizerAuthorizerKeyAuthForCreateMCPService(
                ApiKeys=[
                    InboundAuthorizerAuthorizerKeyAuthApiKeysItemForCreateMCPService(
                        Name=f"mcp-service-api-key-{id}",
                    ),
                ],
            ),
        ),
    ),
    BackendType="Domain",
    BackendConfiguration=BackendForCreateMCPService(
        CustomConfiguration=BackendCustomForCreateMCPService(
            Domain="postman-echo.com",
            ProtocolType="HTTP",
            Port=80,
        )
    ),
    ProjectName="default",
)

mcp_service_id = ""
try:
    response = client.create_mcp_service(request)
    mcp_service_id = response.mcp_service_id
    print(f"MCP service created successfully! MCP service ID: {mcp_service_id}")
except Exception as e:
    print(f"An error occurred: {e}")
    raise e

AgentKit Gateway 部署 MCP 服务需要一段时间。在部署完成后，我们可以检查一下 API 服务 转换的 MCP 服务是否可用。

In [ ]:
import time
from agentkit.sdk.mcp.types import (
    GetMCPServiceRequest,
)

request = GetMCPServiceRequest(
    MCPServiceId=mcp_service_id,
)

endpoint = ""
api_key = ""
try:
    while True:
        response = client.get_mcp_service(request)
        if response.mcp_service.status == "Ready":
            endpoint = response.mcp_service.network_configurations[0].endpoint
            api_key = response.mcp_service.inbound_authorizer_configuration.authorizer.key_auth.api_keys[0].key
            break
        if response.mcp_service.status != "Creating":
            print(f"The MCP service is in {response.mcp_service.status}")
            raise RuntimeError()
        time.sleep(2)
except Exception as e:
    print(f"An error occurred: {e}")
    raise e

url = f"{endpoint}/mcp"
print(f"MCP service URL: {url}")
print(
    f'MCP service API key: {api_key}\n  (Hint: Add a "Authorization: Bearer {api_key}" request header to access the MCP service)'
)

In [ ]:
!pip install fastmcp --quiet

In [ ]:
from fastmcp import Client
from fastmcp.client.transports import StreamableHttpTransport
import json
from mcp.types import TextContent

transport = StreamableHttpTransport(
    url=url,
    headers={
        "Authorization": f"Bearer {api_key}",
    }
)
mcp_client = Client(transport)

async with mcp_client:
    result = await mcp_client.call_tool("getRequest", {
        "foo1": "bar1",
        "foo2": "bar2"
    })
    text: TextContent = result.content[0]
    print(f"The MCP server responds with {json.dumps(json.loads(text.text), indent=2)}.")

恭喜！你已经使用 AgentKit 网关转换 API 服务到 MCP 服务，这个 MCP 服务包含一个名为 `getRequest` 的工具。

## 总结

恭喜你！现在你已经掌握了转换 API 服务到 MCP 服务的方法，

1. 通过 Swagger 或 OpenAPI 3 文档转换 API 服务到 MCP 服务。

你可以继续尝试为 [Swagger Petstore](https://petstore3.swagger.io/)（OpenAPI 3） 和 [HTTPBin](https://httpbin.org/)（Swagger）等样例项目生成 MCP 服务。

现在，请继续学习其他教程。